<a href="https://colab.research.google.com/github/advait-t/IPL_Datasets/blob/main/Fantasy_Score_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall tensorflow
!pip uninstall tensorflow-cpu

Uninstalling tensorflow-cpu-2.4.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/tensorflow/include/external/local_config_python/numpy_include/numpy/random/bitgen.h
    /usr/local/lib/python3.7/dist-packages/tensorflow/include/external/local_config_python/numpy_include/numpy/random/distributions.h
    /usr/local/lib/python3.7/dist-packages/tensorflow/include/tensorflow/core/kernels/collective_nccl.h
    /usr/local/lib/python3.7/dist-packages/tensorflow/include/tensorflow/core/kernels/collective_nccl_broadcaster.h
    /usr/local/lib/python3.7/dist-packages/tensorflow/include/tensorflow/core/kernels/collective_nccl_gatherer.h
    /usr/local/lib/python3.7/dist-packages/tensorflow/include/tensorflow/core/kernels/collective_nccl_reducer.h
    /usr/local/lib/python3.7/dist-packages/tensorflow_cpu-2.4.1.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-cpu-2.4.1


In [ ]:
!pip install tensorflow-cpu

  Using cached https://files.pythonhosted.org/packages/f0/7b/b4d3fbc3ad4d7b5255dff433c06c0105f7815ed584e41f47cb2a00b0c079/tensorflow_cpu-2.4.1-cp37-cp37m-manylinux2010_x86_64.whl


In [ ]:
!unzip '/content/IPL Fantasy Points Predictor.zip'

Archive:  /content/IPL Fantasy Points Predictor.zip
   creating: IPL Fantasy Points Predictor/
  inflating: IPL Fantasy Points Predictor/.DS_Store  
  inflating: __MACOSX/IPL Fantasy Points Predictor/._.DS_Store  
   creating: IPL Fantasy Points Predictor/Bowling/
   creating: IPL Fantasy Points Predictor/Batting/
  inflating: IPL Fantasy Points Predictor/Bowling/.DS_Store  
  inflating: __MACOSX/IPL Fantasy Points Predictor/Bowling/._.DS_Store  
   creating: IPL Fantasy Points Predictor/Bowling/Label_Encoders/
  inflating: IPL Fantasy Points Predictor/Bowling/Scaler.pk  
  inflating: __MACOSX/IPL Fantasy Points Predictor/Bowling/._Scaler.pk  
  inflating: IPL Fantasy Points Predictor/Bowling/Neural_Network_Model.h5  
  inflating: __MACOSX/IPL Fantasy Points Predictor/Bowling/._Neural_Network_Model.h5  
  inflating: IPL Fantasy Points Predictor/Bowling/PCA.pk  
  inflating: __MACOSX/IPL Fantasy Points Predictor/Bowling/._PCA.pk  
  inflating: IPL Fantasy Points Predictor/Bowling/Final_

In [ ]:
from keras.models import load_model
import pickle as pk 
import pandas as pd

def models_importer(folder):
  player = ["Batting","Bowling"]

  for i in player:
    path = folder+'/'+i  

    data = pd.read_csv(path+"/Final_"+i+"_data.csv")
    data = data.drop('Unnamed: 0', axis=1)
    pca = pk.load(open(path+'/PCA.pk','rb'))
    nn = load_model(path+'/Neural_Network_Model.h5')
    scaler = pk.load(open(path+'/Scaler.pk','rb'))
    teams = pk.load(open(path+'/Label_Encoders/Teams.pk','rb'))
    players = pk.load(open(path+'/Label_Encoders/Player.pk','rb'))
    location = pk.load(open(path+'/Label_Encoders/Location.pk','rb'))
    toss_selection = pk.load(open(path+'/Label_Encoders/Toss_Selection.pk','rb'))

    label_encoder = {"teams":teams, "players":players, "location":location, "toss_selection":toss_selection}
    temp = {"data":data, "pca":pca, "nn_model":nn, "scaler":scaler, "label_encoder":label_encoder}

    if i == "Batting":
      bat = temp
    else:
      bowl = temp

  return bat, bowl

In [ ]:
bat, bowl = models_importer("/content/IPL Fantasy Points Predictor")

In [ ]:
bowl

{'data':                  Player  Overs  ...                  Toss_Winner  Choose_to
 0         Praveen Kumar    4.0  ...  Royal Challengers Bangalore      field
 1           Zaheer Khan    4.0  ...  Royal Challengers Bangalore      field
 2         Ashley Noffke    4.0  ...  Royal Challengers Bangalore      field
 3        Jacques Kallis    4.0  ...  Royal Challengers Bangalore      field
 4           Sunil Joshi    3.0  ...  Royal Challengers Bangalore      field
 ...                 ...    ...  ...                          ...        ...
 7613        Isuru Udana    4.0  ...               Mumbai Indians      field
 7614  Washington Sundar    4.0  ...               Mumbai Indians      field
 7615      Navdeep Saini    4.0  ...               Mumbai Indians      field
 7616   Yuzvendra Chahal    4.0  ...               Mumbai Indians      field
 7617         Adam Zampa    4.0  ...               Mumbai Indians      field
 
 [7618 rows x 17 columns],
 'label_encoder': {'location': LabelEnc

# Batsman

In [ ]:
bat['data']

,Player,Runs,Balls,4s,6s,Fantasy_Points,Teams,Opponent_Team,Toss_Winner,Choose_to,Match_Location,strike_rate
0,Sourav Ganguly,10,12,2,0,12,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",83.33
1,Brendon McCullum,158,73,10,13,216,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",216.44
2,Ricky Ponting,20,20,1,1,23,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",100.00
3,David Hussey,12,12,1,0,13,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",100.00
4,Mohammad Hafeez,5,3,1,0,6,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",166.67
...,...,...,...,...,...,...,...,...,...,...,...,...
9755,Suryakumar Yadav,0,2,0,0,-2,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,field,Dubai International Cricket Stadium,0.00
9756,Ishan Kishan,99,58,2,9,125,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,field,Dubai International Cricket Stadium,170.69
9757,Hardik Pandya,15,13,0,1,17,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,field,Dubai International Cricket Stadium,115.38
9758,Kieron Pollard,60,24,3,5,79,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,field,Dubai International Cricket Stadium,250.00


In [ ]:
def player_history(Data, Player, Team, Toss=0, Selection=0, Statium=0):
  Data = Data.drop('Fantasy_Points', axis = 1)
  temp = Data[Data['Player']==Player]
  temp = temp[temp['Opponent_Team']== Team]
  if Toss!=0:
    temp = temp[temp['Toss_Winner']== Toss]
  if Selection!=0:
    temp = temp[temp['Choose_to']==Selection]
  if Statium!=0:
    temp = temp[temp['Match_Location']==Statium]

  return temp

In [ ]:
Data = bat['data']
Player = "Hardik Pandya"
Team = "Royal Challengers Bangalore"
Toss = "Royal Challengers Bangalore"
Selection = "field"
Statium = "M.Chinnaswamy Stadium, Bengaluru"

x = player_history(Data, Player, Team, Toss, Selection, Statium)
x

,Player,Runs,Balls,4s,6s,Teams,Opponent_Team,Toss_Winner,Choose_to,Match_Location,strike_rate
6172,Hardik Pandya,16,6,0,2,Mumbai Indians,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",266.67
8758,Hardik Pandya,32,14,2,3,Mumbai Indians,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",228.57


In [ ]:
def bat_preprocess_input_data(Data, Player, Team, Toss, Selection, Statium):

  x = player_history(Data, Player, Team, Toss, Selection, Statium)

  if x.shape[0] == 0:
    x = player_history(Data, Player, Team, Toss, Selection)
    if x.shape[0] == 0:
      x = player_history(Data, Player, Team, Toss)
      if x.shape[0] == 0:
        x = player_history(Data, Player, Team)

  aruns = x['Runs'].mean()
  aballs = x['Balls'].mean()
  a4 = x['4s'].mean()
  a6 = x['6s'].mean()
  asr =(aruns/aballs)*100
  asr = round(asr, 2)

  Player = bat['label_encoder']['players'].transform([Player])[0]
  Teams = bat['label_encoder']['teams'].transform([list(x['Teams'])[-1]])[0]
  Team = bat['label_encoder']['teams'].transform([Team])[0]
  Toss = bat['label_encoder']['teams'].transform([Toss])[0]
  Selection = bat['label_encoder']['toss_selection'].transform([Selection])[0]
  Statium = bat['label_encoder']['location'].transform([Statium])[0]

  input_test = [Player, aruns, aballs,a4, a6, Teams, Team, Toss, Selection, Statium, asr]
  input_test = pd.DataFrame(input_test).transpose()
  input_test.columns = x.columns

  col_names = ['Runs','Balls','4s','6s','strike_rate']
  features = input_test[col_names]
  features = bat['scaler'].transform(features.values)
  input_test[col_names] = features

  input_test = bat['pca'].transform(input_test)

  return input_test

In [ ]:
input_test = bat_preprocess_input_data(Data, Player, Team, Toss, Selection, Statium)
input_test

array([[ 1.02041731e+02,  1.95280172e+00, -6.47090036e+00,
         3.21912822e+00,  9.39863517e-01,  9.50727000e-01,
         2.25209861e+00,  2.32555398e-01, -2.31475811e-01,
        -5.54264553e-02, -2.28692515e-02]])

In [ ]:
pred = bat['nn_model'].predict(input_test)
pred

array([[35.292313]], dtype=float32)

# Bowler

In [ ]:
bowl['data']

,Player,Overs,Maidens,Runs,Wickets,Econ,Dots,4s,6s,Wd,Nb,Fantasy_Points,Teams,Opponent_Team,Match_Location,Toss_Winner,Choose_to
0,Praveen Kumar,4.0,0,38,0,9.50,13,3,3,1,0,0,Royal Challengers Bangalore,Kolkata Knight Riders,"M.Chinnaswamy Stadium, Bengaluru",Royal Challengers Bangalore,field
1,Zaheer Khan,4.0,0,38,1,9.50,8,3,2,0,0,25,Royal Challengers Bangalore,Kolkata Knight Riders,"M.Chinnaswamy Stadium, Bengaluru",Royal Challengers Bangalore,field
2,Ashley Noffke,4.0,0,40,1,10.00,7,2,2,1,0,25,Royal Challengers Bangalore,Kolkata Knight Riders,"M.Chinnaswamy Stadium, Bengaluru",Royal Challengers Bangalore,field
3,Jacques Kallis,4.0,0,48,1,12.00,9,5,3,1,0,19,Royal Challengers Bangalore,Kolkata Knight Riders,"M.Chinnaswamy Stadium, Bengaluru",Royal Challengers Bangalore,field
4,Sunil Joshi,3.0,0,26,0,8.67,4,0,2,0,0,0,Royal Challengers Bangalore,Kolkata Knight Riders,"M.Chinnaswamy Stadium, Bengaluru",Royal Challengers Bangalore,field
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7613,Isuru Udana,4.0,0,45,2,11.25,7,3,3,3,0,50,Royal Challengers Bangalore,Mumbai Indians,Dubai International Cricket Stadium,Mumbai Indians,field
7614,Washington Sundar,4.0,0,12,1,3.00,13,0,0,0,0,31,Royal Challengers Bangalore,Mumbai Indians,Dubai International Cricket Stadium,Mumbai Indians,field
7615,Navdeep Saini,4.0,0,43,0,10.75,8,1,4,2,0,0,Royal Challengers Bangalore,Mumbai Indians,Dubai International Cricket Stadium,Mumbai Indians,field
7616,Yuzvendra Chahal,4.0,0,48,1,12.00,2,1,4,0,0,19,Royal Challengers Bangalore,Mumbai Indians,Dubai International Cricket Stadium,Mumbai Indians,field


In [ ]:
def player_history(Data, Player, Team, Toss=0, Selection=0, Statium=0):
  Data = Data.drop('Fantasy_Points', axis = 1)
  temp = Data[Data['Player']==Player]
  temp = temp[temp['Opponent_Team']== Team]
  if Toss!=0:
    temp = temp[temp['Toss_Winner']== Toss]
  if Selection!=0:
    temp = temp[temp['Choose_to']==Selection]
  if Statium!=0:
    temp = temp[temp['Match_Location']==Statium]

  return temp

In [ ]:
Data = bowl['data']
Player = "Hardik Pandya"
Team = "Royal Challengers Bangalore"
Toss = "Royal Challengers Bangalore"
Selection = "field"
Statium = "M.Chinnaswamy Stadium, Bengaluru"

x = player_history(Data, Player, Team, Toss, Selection, Statium)
x

,Player,Overs,Maidens,Runs,Wickets,Econ,Dots,4s,6s,Wd,Nb,Teams,Opponent_Team,Match_Location,Toss_Winner,Choose_to
4827,Hardik Pandya,3.0,0,37,0,12.33,4,3,2,2,0,Mumbai Indians,Royal Challengers Bangalore,"M.Chinnaswamy Stadium, Bengaluru",Royal Challengers Bangalore,field
6834,Hardik Pandya,3.0,0,37,0,12.33,6,5,2,0,0,Mumbai Indians,Royal Challengers Bangalore,"M.Chinnaswamy Stadium, Bengaluru",Royal Challengers Bangalore,field


In [ ]:
def bowl_preprocess_input_data(Data, Player, Team, Toss, Selection, Statium):

  x = player_history(Data, Player, Team, Toss, Selection, Statium)

  if x.shape[0] == 0:
    x = player_history(Data, Player, Team, Toss, Selection)
    if x.shape[0] == 0:
      x = player_history(Data, Player, Team, Toss)
      if x.shape[0] == 0:
        x = player_history(Data, Player, Team)

  
  aovers = x['Overs'].mean()
  amaidens = x['Maidens'].mean()
  aruns = x['Runs'].mean()
  awickets = x['Wickets'].mean()
  aecon = x['Econ'].mean()
  adots = x['Dots'].mean()
  a4 = x['4s'].mean()
  a6 = x['6s'].mean()
  awd = x['Wd'].mean()
  anb = x['Nb'].mean()

  Player = bowl['label_encoder']['players'].transform([Player])[0]
  Teams = bowl['label_encoder']['teams'].transform([list(x['Teams'])[-1]])[0]
  Team = bowl['label_encoder']['teams'].transform([Team])[0]
  Toss = bowl['label_encoder']['teams'].transform([Toss])[0]
  Selection = bowl['label_encoder']['toss_selection'].transform([Selection])[0]
  Statium = bowl['label_encoder']['location'].transform([Statium])[0]

  input_test = [Player, aovers, amaidens, aruns, awickets, aecon, adots, a4, a6, awd, anb, Teams, Team, Toss, Selection, Statium]
  input_test = pd.DataFrame(input_test).transpose()
  input_test.columns = x.columns

  col_names = ['Overs','Maidens','Runs','Wickets','Econ','Dots','4s','6s','Wd','Nb']
  features = input_test[col_names]
  features = bowl['scaler'].transform(features.values)
  input_test[col_names] = features

  input_test = bowl['pca'].transform(input_test)

  return input_test

In [ ]:
input_test = bowl_preprocess_input_data(Data, Player, Team, Toss, Selection, Statium)

In [ ]:
pred = bowl['nn_model'].predict(input_test)
pred

array([[0.2264725]], dtype=float32)

In [ ]:
Player = "MS Dhoni"
Team = "Royal Challengers Bangalore"
Toss = "Chennai Super Kings"
Selection = "bat"
Statium = "M.Chinnaswamy Stadium, Bengaluru"

batsmen = list(bat['data']['Player'].unique())
bowler = list(bowl['data']['Player'].unique())

if Player in batsmen:
  input_test = bat_preprocess_input_data(bat['data'], Player, Team, Toss, Selection, Statium)
  predbt = bat['nn_model'].predict(input_test)
if Player in bowler:
  input_test = bowl_preprocess_input_data(bowl['data'], Player, Team, Toss, Selection, Statium)
  predbl = bowl['nn_model'].predict(input_test)

pred = predbt+predbl
predbt, predbl, pred

(array([[86.20565]], dtype=float32),
 array([[21.702696]], dtype=float32),
 array([[107.90835]], dtype=float32))

In [ ]:
import sklearn
sklearn.__version__

'0.22.2.post1'

# IPL Simulation

In [ ]:
bat['data']

,Player,Runs,Balls,4s,6s,Fantasy_Points,Teams,Opponent_Team,Toss_Winner,Choose_to,Match_Location,strike_rate
0,Sourav Ganguly,10,12,2,0,12,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",83.33
1,Brendon McCullum,158,73,10,13,216,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",216.44
2,Ricky Ponting,20,20,1,1,23,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",100.00
3,David Hussey,12,12,1,0,13,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",100.00
4,Mohammad Hafeez,5,3,1,0,6,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru",166.67
...,...,...,...,...,...,...,...,...,...,...,...,...
9755,Suryakumar Yadav,0,2,0,0,-2,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,field,Dubai International Cricket Stadium,0.00
9756,Ishan Kishan,99,58,2,9,125,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,field,Dubai International Cricket Stadium,170.69
9757,Hardik Pandya,15,13,0,1,17,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,field,Dubai International Cricket Stadium,115.38
9758,Kieron Pollard,60,24,3,5,79,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,field,Dubai International Cricket Stadium,250.00


In [ ]:
bat_data = pd.read_csv("https://raw.githubusercontent.com/advait-t/IPL_Datasets/main/Data/Batting_updated%20.csv")
bat_data = bat_data.iloc[:,1:]
bat_data.head()

,Player,Description,Runs,Balls,4s,6s,SR,Team,Match_id,Season,Fantasy_Points,Teams,Opponent_Team,Toss_Winner,Choose_to,Match_Location
0,Sourav Ganguly,c Kallis b Khan,10,12,2,0,83.33,1,335982,1,12,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru"
1,Brendon McCullum,not out,158,73,10,13,216.44,1,335982,1,216,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru"
2,Ricky Ponting,c Kumar b Kallis,20,20,1,1,100.00,1,335982,1,23,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru"
3,David Hussey,c White b Noffke,12,12,1,0,100.00,1,335982,1,13,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru"
4,Mohammad Hafeez,not out,5,3,1,0,166.67,1,335982,1,6,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,"M.Chinnaswamy Stadium, Bengaluru"


In [ ]:
def ground_history(Data, Stadium, Team=0):
  Data = Data.drop('Fantasy_Points', axis = 1)
  temp = Data[Data['Match_Location']==Stadium]
  if Team!=0:
    temp = temp[temp['Teams']==Team]

  temp = temp.drop(['Description','SR','Match_id','Toss_Winner','Choose_to'], axis=1)
  return temp

In [ ]:
ground_history(bat_data, 'Wankhede Stadium, Mumbai')

,Player,Runs,Balls,4s,6s,Team,Season,Teams,Opponent_Team,Match_Location
42,Luke Ronchi,8,10,2,0,1,1,Mumbai Indians,Royal Challengers Bangalore,"Wankhede Stadium, Mumbai"
43,Sanath Jayasuriya,29,16,5,1,1,1,Mumbai Indians,Royal Challengers Bangalore,"Wankhede Stadium, Mumbai"
44,Dominic Thornely,0,5,0,0,1,1,Mumbai Indians,Royal Challengers Bangalore,"Wankhede Stadium, Mumbai"
45,Robin Uthappa,48,38,5,1,1,1,Mumbai Indians,Royal Challengers Bangalore,"Wankhede Stadium, Mumbai"
46,Pinal Shah,19,23,2,0,1,1,Mumbai Indians,Royal Challengers Bangalore,"Wankhede Stadium, Mumbai"
...,...,...,...,...,...,...,...,...,...,...
8905,Ravindra Jadeja,1,2,0,0,2,12,Chennai Super Kings,Mumbai Indians,"Wankhede Stadium, Mumbai"
8906,Dwayne Bravo,8,9,0,1,2,12,Chennai Super Kings,Mumbai Indians,"Wankhede Stadium, Mumbai"
8907,Deepak Chahar,7,7,1,0,2,12,Chennai Super Kings,Mumbai Indians,"Wankhede Stadium, Mumbai"
8908,Shardul Thakur,12,5,1,1,2,12,Chennai Super Kings,Mumbai Indians,"Wankhede Stadium, Mumbai"


In [ ]:
def bat_preprocess_input_data(Data, Stadium, Team=0):

  data = ground_history(Data, 'Wankhede Stadium, Mumbai')
  players = data['Player'].unique()
  master_list = []

  for i in players:
    x = player_history(Data, i, Stadium, Team=0, Toss=0, Selection=0)
    if Team==0:
      t = x['Opponent_Team'].unique()
      for j in t:
        temp = x[x['Opponent_Team']==j]
        aruns = temp['Runs'].sum()
        aballs = temp['Balls'].sum()
        a4 = temp['4s'].sum()
        a6 = temp['6s'].sum()
        #asr =(aruns/aballs)*100
        #asr = round(asr, 2)
        
        input_test = [i, aruns, aballs,a4, a6, j, Stadium]
        #print(input_test)
        master_list.append(input_test)
  master_list = pd.DataFrame(master_list)
  master_list.columns = ["Player",'Runs','Balls','4s','6s','Opponent_Team','Match_location']
  return master_list


In [ ]:
bat_data.groupby('Player')['Runs'].sum()

Player
AB de Villiers      2621020721105215050504044117928626715112274545...
Aakash Chopra                                               224131211
Aaron Finch         2181125030302564067166418371515455102198844165...
Abdul Samad                                                7023812208
Abdur Razzak                                                        0
                                          ...                        
Yusuf Abdulla                                                       0
Yusuf Pathan        1076121558168839486245561142106220241222761719...
Yuvraj Singh        2357184014171249237172741638302105014485818121...
Yuzvendra Chahal                                        0131014404110
Zaheer Khan                         317801140240202301021110007014121
Name: Runs, Length: 521, dtype: object

In [ ]:
Team, Data, Stadium, i = 0, bat_data, 'Wankhede Stadium, Mumbai', 'Virat Kohli'

x = player_history(Data, i, Stadium, Team=0, Toss=0, Selection=0)
if Team==0:
  t = x['Opponent_Team'].unique()
  for j in t:
    temp = x[x['Opponent_Team']==j]
    print(temp)

           Player Runs  ...   Opponent_Team            Match_Location
54    Virat Kohli   23  ...  Mumbai Indians  Wankhede Stadium, Mumbai
4617  Virat Kohli   36  ...  Mumbai Indians  Wankhede Stadium, Mumbai
5255  Virat Kohli    1  ...  Mumbai Indians  Wankhede Stadium, Mumbai
6551  Virat Kohli   33  ...  Mumbai Indians  Wankhede Stadium, Mumbai
7354  Virat Kohli   20  ...  Mumbai Indians  Wankhede Stadium, Mumbai
7937  Virat Kohli   92  ...  Mumbai Indians  Wankhede Stadium, Mumbai

[6 rows x 10 columns]
           Player Runs  ...        Opponent_Team            Match_Location
3751  Virat Kohli   70  ...  Chennai Super Kings  Wankhede Stadium, Mumbai

[1 rows x 10 columns]


In [ ]:
bat_preprocess_input_data(bat_data, 'Wankhede Stadium, Mumbai', Team=0).head(50)

,Player,Runs,Balls,4s,6s,Opponent_Team,Match_location
0,Luke Ronchi,8,10,2,0,Royal Challengers Bangalore,"Wankhede Stadium, Mumbai"
1,Sanath Jayasuriya,29,16,5,1,Royal Challengers Bangalore,"Wankhede Stadium, Mumbai"
2,Sanath Jayasuriya,114,48,9,11,Chennai Super Kings,"Wankhede Stadium, Mumbai"
3,Sanath Jayasuriya,48,17,6,3,Kolkata Knight Riders,"Wankhede Stadium, Mumbai"
4,Sanath Jayasuriya,20,12,1,2,Punjab Kings,"Wankhede Stadium, Mumbai"
5,Dominic Thornely,0,5,0,0,Royal Challengers Bangalore,"Wankhede Stadium, Mumbai"
6,Dominic Thornely,1,6,0,0,Kolkata Knight Riders,"Wankhede Stadium, Mumbai"
7,Robin Uthappa,48,38,5,1,Royal Challengers Bangalore,"Wankhede Stadium, Mumbai"
8,Robin Uthappa,16,19,2,0,Chennai Super Kings,"Wankhede Stadium, Mumbai"
9,Robin Uthappa,9,8,2,0,Kolkata Knight Riders,"Wankhede Stadium, Mumbai"


In [ ]:
def player_history(Data, Player, Stadium, Team=0, Toss=0, Selection=0):
  Data = Data.drop('Fantasy_Points', axis = 1)
  temp = Data[Data['Player']==Player]
  temp = temp[temp['Match_Location']==Stadium]
  
  if Toss!=0:
    temp = temp[temp['Toss_Winner']== Toss]
  if Selection!=0:
    temp = temp[temp['Choose_to']==Selection]
  if Team!=0:
    temp = temp[temp['Opponent_Team']== Team]
  
  temp = temp.drop(['Description','SR','Match_id','Toss_Winner','Choose_to'], axis=1)

  return temp

In [ ]:
player_history(bat_data, 'Rohit Sharma', 'Wankhede Stadium, Mumbai')

,Player,Runs,Balls,4s,6s,Team,Season,Teams,Opponent_Team,Match_Location
3009,Rohit Sharma,20,18,1,1,2,4,Mumbai Indians,Pune Warriors,"Wankhede Stadium, Mumbai"
3051,Rohit Sharma,87,48,8,5,1,4,Mumbai Indians,Chennai Super Kings,"Wankhede Stadium, Mumbai"
3301,Rohit Sharma,18,11,3,0,1,4,Mumbai Indians,Punjab Kings,"Wankhede Stadium, Mumbai"
3431,Rohit Sharma,49,32,2,3,1,4,Mumbai Indians,Delhi Capitals,"Wankhede Stadium, Mumbai"
3593,Rohit Sharma,4,6,1,0,2,4,Mumbai Indians,Deccan Chargers,"Wankhede Stadium, Mumbai"
3694,Rohit Sharma,58,47,5,1,1,4,Mumbai Indians,Rajasthan Royals,"Wankhede Stadium, Mumbai"
3772,Rohit Sharma,0,0,0,0,2,4,Mumbai Indians,Kolkata Knight Riders,"Wankhede Stadium, Mumbai"
3853,Rohit Sharma,1,6,0,0,2,5,Mumbai Indians,Pune Warriors,"Wankhede Stadium, Mumbai"
4005,Rohit Sharma,21,13,2,1,1,5,Mumbai Indians,Rajasthan Royals,"Wankhede Stadium, Mumbai"
4135,Rohit Sharma,29,27,5,0,1,5,Mumbai Indians,Delhi Capitals,"Wankhede Stadium, Mumbai"


In [ ]:
aruns = x['Runs'].mean()
  aballs = x['Balls'].mean()
  a4 = x['4s'].mean()
  a6 = x['6s'].mean()
  asr =(aruns/aballs)*100
  asr = round(asr, 2)

  input_test = [Player, aruns, aballs,a4, a6, Teams, Team, Toss, Selection, Statium, asr]
  input_test = pd.DataFrame(input_test).transpose()
  input_test.columns = x.columns

  col_names = ['Runs','Balls','4s','6s','strike_rate']
  features = input_test[col_names]
  features = bat['scaler'].transform(features.values)
  input_test[col_names] = features

In [ ]:
import requests
import time
import datetime
from datatime import datetime
import pandas_datareader as web
#import yfinance as yf

symbol='ITC.NS'
dt = datetime.now()
aapl= web.DataReader(symbol, data_source='yahoo', start = dt, end = dt)
price=(aapl["Close"][0]).round(2)
price=str(price)

ModuleNotFoundError: ignored

In [ ]:
price

'241.45'

In [ ]:
from datetime import datetime, timedelta
from dateutil.parser import parse

def DATE_CONVERTER(date):
  temp1 = parse(date)
  temp2 = temp1.strftime('%Y-%m-%d')
  return temp2

DATE_CONVERTER("2019/12/14")

'2019-12-14'

In [ ]:
aapl=pdr.get_data_yahoo(symbol,  start = 

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)	

now = 2021-04-15 09:00:32.393162
date and time = 15/04/2021 09:00:32


In [ ]:
!pip install alpha_vantage

     |████████████████████████████████| 1.3MB 18.3MB/s 
     |████████████████████████████████| 296kB 37.3MB/s 
     |████████████████████████████████| 143kB 41.6MB/s 


In [ ]:
import pandas as pd #data manipulation and analysis package
from alpha_vantage.timeseries import TimeSeries #enables data pull from Alpha Vantage
import matplotlib.pyplot as plt #if you want to plot your findings
import time

In [ ]:
#Getting the data from alpha_vantage
ts = TimeSeries(key='P21PK2V95U3POW0R', output_format='pandas')
data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')

In [ ]:
data

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2021-04-14 19:50:00,255.38,255.38,255.38,255.38,1075.0
2021-04-14 19:49:00,255.65,255.65,255.41,255.41,1520.0
2021-04-14 19:48:00,255.65,255.65,255.65,255.65,299.0
2021-04-14 19:38:00,255.80,255.80,255.80,255.80,100.0
2021-04-14 19:28:00,255.65,255.65,255.65,255.65,105.0
...,...,...,...,...,...
2021-04-01 04:06:00,238.82,238.82,238.82,238.82,480.0
2021-04-01 04:05:00,238.80,238.80,238.80,238.80,293.0
2021-04-01 04:04:00,238.97,238.97,238.71,238.71,255.0


In [ ]:
close_data = data['4. close'] #The close data column
last_price = close_data[0] #Selecting the last price from the close_data column
#Check if you're getting a correct value
print(last_price) 

255.38


In [ ]:
import pandas as pd #data manipulation and analysis package
from alpha_vantage.timeseries import TimeSeries #enables data pull from Alpha Vantage
import matplotlib.pyplot as plt #if you want to plot your findings
import time
import smtplib #enables you to send emails

In [ ]:
#Getting the data from alpha_vantage
ts = TimeSeries(key='P21PK2V95U3POW0R', output_format='pandas')
data, meta_data = ts.get_intraday(symbol='DOGE',interval='1min', outputsize='full')

ValueError: ignored

In [ ]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies

cy = CryptoCurrencies(key='P21PK2V95U3POW0R', output_format='pandas')

In [ ]:
x = cy.get_digital_currency_exchange_rate(from_currency="DOGE", to_currency="INR")

In [ ]:
x[0]['5. Exchange Rate'][0]

'17.54038609'

In [ ]:
price = ''
price = x[0]['5. Exchange Rate'][0]
price=float(price)
16 >= price or price >= 17

True

In [ ]:
close_data = data['4. close'] #The close data column
last_price = close_data[0] #Selecting the last price from the close_data column
#Check if you're getting a correct value
print(last_price) 

255.38


In [ ]:
#Set the desired message you want to see once the stock price is at a certain level
sender_email = "limitstockpricenotifier@gmail.com" #The sender email
rec_email = "jatprajapati753@gmail.com" #The receiver email
password = ("limit@notifier") #The password to the sender email
message = "MSFT STOCK ALERT!!! The stock is at above price you set " + "%.6f" % last_price  #The message you want to send

In [ ]:
target_sell_price = 220 #enter the price you want to sell at
if last_price > target_sell_price:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, password) #logs into your email account
    print("Login Success") #confirms that you have logged in succesfully
    server.sendmail(sender_email, rec_email, message) #send the email with your custom mesage
    print("Email was sent") #confirms that the email was sent 

SMTPAuthenticationError: ignored